In [67]:
#import the necassary libriaries
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Flatten,Input
from sklearn.metrics import confusion_matrix
from os import listdir
from tensorflow.keras import layers
import pandas as pd


In [68]:
#import the content from the google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
#Read the train data to from the listdir
data_list_model1=listdir('/content/drive/MyDrive/chest_xray/train')



In [70]:
#Read the train and test data from the google drive
Data_set_model2='/content/drive/MyDrive/chest_xray/val'
train_dir='/content/drive/MyDrive/chest_xray/train'
test_dir='/content/drive/MyDrive/chest_xray/test'

In [71]:
#create the image shape variable with image size 150,150
IMAGE_SIZE=(150,150)
#create the class
NUM_CLASSES=len(data_list_model1)
#Ceate_batch size with bacth number
BATCH_SIZE=10
#Create epochs
NUM_EPOCHS=100
#Create learning rate
LEARNING_RATE=0.0001


In [72]:
#Train datagen
train_datagen=ImageDataGenerator(rescale=1./255,rotation_range=50,featurewise_center=True,featurewise_std_normalization=True,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.25,zoom_range=0.1,zca_whitening=True,channel_shift_range=20,horizontal_flip=True,vertical_flip=True,fill_mode='constant')

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:342: UserWarning: This ImageDataGenerator specifies `zca_whitening` which overrides setting of`featurewise_std_normalization`.
  warnings.warn('This ImageDataGenerator specifies '


In [73]:
#Trian data
train_batches=train_datagen.flow_from_directory(Data_set_model2,target_size=IMAGE_SIZE,shuffle=True,batch_size=BATCH_SIZE,subset='training',seed=42,class_mode='categorical')

Found 16 images belonging to 2 classes.


In [74]:
#Validation
Validation_batches=train_datagen.flow_from_directory(Data_set_model2,target_size=IMAGE_SIZE,shuffle=True,batch_size=BATCH_SIZE,subset='validation',seed=42,class_mode='categorical')

Found 0 images belonging to 2 classes.


In [75]:
#Create vgg16 model for detection the x_ray  with the paramenters
vgg16_model=tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=Input(shape=(150,150,3))
)


In [76]:
#Create a for loop for vgg16 model and trainble
for layer in vgg16_model.layers[:-4]:
  layer.trainable=False
#Install the Sequental model 
model2c=tf.keras.Sequential()
#add the vgg16 model to the sequential
model2c.add(vgg16_model)
#Faltte the data
model2c.add(Flatten())
#Add the dense layer with 64, and relu activation function
model2c.add(Dense(64,activation='relu'))
#Add the another dense layer with 2, and add activation softmax function
model2c.add(layers.Dense(2,activation='softmax'))


In [77]:
#Complie the model with categroical_crossentropt with adam optimers and accuracyscore
model2c.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE),metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [78]:
#Create the train varablw with multplication train_batcehes with n and divided with train_batch and multplication with batch size same as test data
STEP_SIZE_TRAIN=train_batches.n//train_batches.batch_size
STEP_SIZE_TEST=Validation_batches.n//Validation_batches.batch_size

In [79]:
#fit the model to the trian data with epochs 5 
model2c.fit_generator(train_batches,steps_per_epoch=STEP_SIZE_TRAIN,epochs=5,verbose=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:739: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/5
1/1 [==============================] - 1s 786ms/step - loss: 0.6425 - accuracy: 0.5000
Epoch 2/5
1/1 [==============================] - 0s 231ms/step - loss: 1.0371 - accuracy: 0.4000
Epoch 3/5
1/1 [==============================] - 0s 212ms/step - loss: 0.6071 - accuracy: 0.6000
Epoch 4/5
1/1 [==============================] - 0s 222ms/step - loss: 0.6597 - accuracy: 0.7000
Epoch 5/5
1/1 [==============================] - 0s 230ms/step - loss: 0.5998 - accuracy: 0.7000


In [80]:
#rescale the image 
train_DataGenerator=ImageDataGenerator(rescale=1./255)

In [81]:
#Create the train data with parameters
Train_bactches=train_DataGenerator.flow_from_directory(train_dir,target_size=IMAGE_SIZE,batch_size=1,shuffle=False,seed=42,class_mode='categorical')
Train_bactches.reset()

Found 776 images belonging to 2 classes.


In [82]:
#And evaluate the model with tain data and steps
TrainResult=model2c.evaluate_generator(Train_bactches,steps=len(Train_bactches),use_multiprocessing=False,verbose=1,workers=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  


776/776 [==============================] - 22s 28ms/step - loss: 0.6709 - accuracy: 0.5722


In [83]:
#Print the train loss and accuracy score
print('Train Loss:',TrainResult[0],'Train accuracy:',TrainResult[1])

Train Loss: 0.6709201335906982 Train accuracy: 0.5721649527549744


In [84]:
#Predictions
prediction=model2c.predict_generator(Train_bactches,steps=len(Train_bactches))
prediction_class=np.argmax(prediction,axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  


In [85]:
#Let's print the confusion matrix
print(confusion_matrix(Train_bactches.classes,prediction_class))

[[444 332]
 [  0   0]]


In [86]:
#Again rescale the test data
TEST_DataGenerator=ImageDataGenerator(rescale=1./255)

In [65]:
#Create the train data with parameters
TEST_batches=TEST_DataGenerator.flow_from_directory(test_dir,target_size=IMAGE_SIZE,batch_size=5,shuffle=False,seed=42,class_mode='categorical')
#Reset the model
TEST_batches.reset()
#evaluate the generator with len test batches
TESTResult=model2c.evaluate_generator(TEST_batches,steps=len(TEST_batches),use_multiprocessing=False,verbose=1,workers=1)
#print the Test loss and accuracy score
print('Test Loss:',TESTResult[0],'Test accuracy:',TESTResult[1])
#train accuray_score
prediction=model2c.predict_generator(TEST_batches,steps=len(TEST_batches))
prediction_class=np.argmax(prediction,axis=1)

Found 624 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  


125/125 [==============================] - 8s 60ms/step - loss: 0.5759 - accuracy: 0.6587
Test Loss: 0.5759245157241821 Test accuracy: 0.6586538553237915


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  # Remove the CWD from sys.path while we load stuff.


In [66]:
#Let's print the confusion matrix
print(confusion_matrix(TEST_batches.classes,prediction_class))

[[ 21 213]
 [  0 390]]
